# Scraping Analysis

In [1]:
# Imports
import math
import numpy as np
import pandas as pd
import geopy.distance
from ipysheet import from_dataframe
from ftplib import FTP
import fileinput
from io import BytesIO
import pickle
import re
from tqdm import tqdm
from IPython.core.display import display
import IPython

#regrssion
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

# scaling
from sklearn.preprocessing import scale, MinMaxScaler

# Plots
import matplotlib.pyplot as plt


### Read Data

In [2]:
current_db = 'zip_test_8'
site_url = 'giow1012.siteground.us'
port = 21
password = 'Mooose33'
username = 'michaelort@ortpropertygroup.com'
site_folder = '/scrapes/final_database_frames'


def read_current_db(site_url, port, username, password, site_folder,
                    current_db):
    # connect
    ftp = FTP()
    ftp.set_debuglevel(2)
    ftp.connect(site_url, port)
    ftp.login(username, password)
    ftp.cwd(site_folder)
    # read
    r = BytesIO()
    ftp.retrbinary('RETR {}'.format(current_db + '.pkl'), r.write)
    r.seek(0)
    old = pickle.load(r)
    r.close
    ftp.close()
    return old


#df = read_current_db(site_url, port, username, password, site_folder, current_db)
#df.to_pickle('../housing_data/data/final_database_frames/zip_test_7.pkl')
df = pd.read_pickle('../housing_data/data/final_database_frames/' + current_db +
                    '.pkl')

# change name of city to scraped city so that i can have my own city column extracted
# from state_city zip
df = df.rename(columns={'city': 'scraped_city'})


### Read third party data

In [3]:
location_data = pd.read_csv(
    '../housing_data/data/geographic_data/zip_code_database_cleaned.csv',
    converters={'zip': lambda x: str(x)})

# get population by zip
pop_data = pd.read_pickle('../housing_data/data/geographic_data/population_zipcode.pkl')

def add_pop_data_to_location_data(pop_data, location_data):
    # Add pop data
    location_data = location_data.merge(pop_data,left_on='zip',right_on = 'zip_code',how='left')
    return location_data

location_data = add_pop_data_to_location_data(pop_data, location_data)

def get_location_data(location_data):
    # read data
    location_data = location_data.loc[:, [
        'state_city_zip', 'latitude', 'longitude', 'county','pop'
    ]]
    # create state_city
    location_data['state_city'] = location_data.state_city_zip.str.replace(
        '\/\d+', '')

    unique_city_state_location_data = location_data.drop_duplicates(
        subset=['state_city'], keep='last')

    unique_city_state_location_data = unique_city_state_location_data.drop(
        ['state_city_zip','pop'], axis=1)
    unique_city_state_location_data = unique_city_state_location_data.rename(columns={'state_city':'state_city_zip'})
    city_pops = location_data[['state_city','pop']].groupby(['state_city'],as_index=True).sum()
    city_pops.reset_index(level=0,inplace=True)
    city_pops = city_pops.rename(columns={'state_city':'state_city_zip'})

    unique_city_state_location_data = unique_city_state_location_data.merge(city_pops,on='state_city_zip')
    location_data = location_data.drop(['state_city'], axis=1)

    return location_data, unique_city_state_location_data


location_data, unique_city_state_location_data = get_location_data(
    location_data)

hotel_data = pd.read_csv(
    '../housing_data/2012_census_hotel_accomodation/ECN_2012_US_72Z1_with_ann.csv',
    converters={'GEO.id2': lambda x: str(x)},
    header=0)


def prep_hotel_data(hotel_data):
    # filter for all establishments
    hotel_data = hotel_data.iloc[1:, :]
    hotel_data = hotel_data[hotel_data['RCPSZFE.display-label'] ==
                            'All establishments']
    hotel_data = hotel_data.drop_duplicates(
        ['GEO.id2', 'NAICS.display-label', 'ESTAB'], keep='last')
    hotel_data = hotel_data.pivot(index='GEO.id2',
                                  columns='NAICS.display-label',
                                  values='ESTAB')
    hotel_data = hotel_data[[
        'Accommodation', 'Bed-and-breakfast inns', 'Casino hotels',
        'Hotels (except casino hotels) and motels', 'Traveler accommodation',
        'Recreational and vacation camps (except campgrounds)'
    ]]
    return hotel_data


hotel_data = prep_hotel_data(hotel_data)

# read zillow data
zillow = pd.read_pickle(
    '../housing_data/data/zillow/prepped_zillow_data/zillow_19-06.pkl')
zillow_city = pd.read_pickle(
    '../housing_data/data/zillow/prepped_zillow_data/zillow_19-06_City_.pkl')
# change zillow_city to state city zip because that is id column for both cities and zips here

zillow_city = zillow_city.rename(columns={'state_city':'state_city_zip'})


# read hud data
hud = pd.read_pickle(
    '../housing_data/data/real_estate/HUD_zip_40_percentile.pkl')

# listing data for reviews
#listing_data = pd.read_json('../housing_data/data/listing_data/scraped_listings_9_27.json')

# Read search results
search_results = pd.read_csv(
    '../housing_data/data/search_results/research_results.csv',converters={'zip': lambda x: str(x)})

def isNaN(num):
    return num != num


def search_results_clean(col):
    col = col.str.lower()
    col = col.str.replace(' ','-')
    return col

def prep_search_results(search_results):
    search_results = search_results.dropna(subset=['city'])
    search_results.loc[:,['state','city']] = search_results[['state','city']].apply(search_results_clean, axis=0)
    state_city_zips = []
    for row in search_results.iterrows():
        state_city_zip = ''
        if not row[1]['state']=='':
            state_city_zip += row[1]['state']
            if not row[1]['city']=='':
                state_city_zip += '/'+row[1]['city']
                if not row[1]['zip']=='':
                    state_city_zip += '/'+row[1]['zip'] 
        state_city_zips.append(state_city_zip)
    search_results['state_city_zip'] = state_city_zips
    return search_results[['legal_ranking', 'notes', 'state_city_zip','finalists']]


search_results = prep_search_results(search_results)

df = df.merge(search_results, on = 'state_city_zip',how = 'left')


/home/michael/anaconda3/envs/airbnb_analysis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,5,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Merging Zillow, Hotel, and Zip Census Data

In [4]:
def add_columns_for_merging(df):
    df['zip'] = df.state_city_zip.str.extract(r'(\d+)')
    df['zip_data'] = df.state_city_zip.str.contains(r'\d', regex=True)
    df['state_city'] = df.state_city_zip.str.replace('\/\d+', '')
    return df


df = add_columns_for_merging(df)


def add_zillow_data(df, zillow, zillow_city):
    city_data = df[~df.zip_data].reset_index().merge(
        zillow_city, on='state_city_zip',how='left').set_index('index')
    zip_data = df[df.zip_data].reset_index().merge(
        zillow, on='state_city_zip', how='left').set_index('index')
    df = pd.concat([city_data, zip_data], axis=0).sort_index()

    return df


df = add_zillow_data(df, zillow, zillow_city)

df = df.merge(hotel_data, left_on='zip', right_on='GEO.id2', how='left')

# This is really to add location data
df = add_zillow_data(df, location_data, unique_city_state_location_data)


def add_hud_estimate(hud, zillow):
    zillow['zip'] = zillow.state_city_zip.str.extract(r'(\d+)')
    hud['median_rent_hud'] = hud.iloc[:, 3:].mean(axis=1)
    zillow_hud = hud[['zcta', 'median_rent_hud']].merge(zillow[['zip', 'Zri']],
                                                        left_on='zcta',
                                                        right_on='zip',
                                                        how='inner')
    zillow_hud = zillow_hud.dropna()
    lin_reg_mod = linear_model.LinearRegression()
    lin_reg_mod.fit(zillow_hud['median_rent_hud'].values.reshape(-1, 1),
                    zillow_hud['Zri'].values.reshape(-1, 1))
    hud['adjusted_median_rent_hud'] = lin_reg_mod.predict(
        hud.median_rent_hud.values.reshape(-1, 1))
    return hud


def add_hud_data(df, hud):
    hud = hud[['zcta', 'median_rent_hud', 'adjusted_median_rent_hud']].merge(df[['zip', 'state_city_zip', 'state_city']],
                    left_on='zcta',
                    right_on='zip')
    hud_city = hud[['state_city', 'median_rent_hud', 'adjusted_median_rent_hud']].groupby(['state_city'],as_index=False).mean()
    hud_city = hud_city.rename(columns={'state_city':'state_city_zip'})

        
    hud = hud[['state_city_zip', 'median_rent_hud', 'adjusted_median_rent_hud']]
    df = add_zillow_data(df, hud, hud_city)
    return df


# add hud estimated rent
hud = add_hud_estimate(hud, zillow)

# add hud data
df = add_hud_data(df, hud)



### Extrapolating columns

In [5]:
def get_history_extrapolations(my_history):
    avg = []
    cv = []

    for i in my_history:
        if type(i) is list:
            if len(i) > 0:
                avg.append(np.average(i))
                cv.append(np.std(i) / np.average(i))
            else:
                avg.append(np.nan)
                cv.append(np.nan)
        else:
            avg.append(np.nan)
            cv.append(np.nan)
    return avg, cv


def get_num_zip_codes(df):
    freq_table = df.state_city.value_counts()
    num_times = []
    for i in df.state_city:
        num_times.append(freq_table.loc[i])
    return num_times


def add_city_column(df):
    my_cities_zip = df[df.zip_data].state_city_zip.str.extract('/(.*?)/')
    my_cities_city = df[~df.zip_data].state_city_zip.str.extract('/([^/]*)')
    city = pd.concat([my_cities_city, my_cities_zip], axis=0).sort_index()
    return city


def get_distance_from_nyc(long_lat_row):
    if np.isnan(long_lat_row[1]) or np.isnan(long_lat_row[0]):
        distance = np.nan
    else:
        new_york_coords = (40.812522, -73.951924)
        place = (long_lat_row[1], long_lat_row[0])
        distance = geopy.distance.vincenty(place, new_york_coords).mi
    return distance



def get_annual_change(df):
    last_choices = ['q1','q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9','q10', 'q11', 'q12','q13']
    df_growth = df[last_choices]
    latest_values = []
    old_values = []
    for row in df_growth.iterrows():
        latest_value = np.nan
        old_value = np.nan
        for idx, i in enumerate(last_choices):
            if not isNaN(row[1][idx]):
                if idx >4:
                    if not isNaN(row[1][idx-4]):
                        latest_value = float(row[1][idx])
                        old_value = float(row[1][idx-4])
                
        latest_values.append(latest_value)
        old_values.append(old_value)
    
    estimates = [round(((105.5-i)-(105.5-k))/(105.5-k),3) if ((not isNaN(k)) and (not isNaN(i))) else np.nan for i,k in zip(latest_values,old_values)]
    return estimates    


def get_scores(df):
    df_scaled = df[[
            'current_active_listings', 'avg_monthly_revenue',
            'seasonality_monthly_revenue', 'my_mean_rent_to_rent',
            'distance_from_Eli', 'avg_monthly_occupancy', 'rating_avg','annual_growth_rate'
        ]].apply(lambda x: scale(x),axis=0)
    
    weights = np.array([1,1,-0.5,2,-1,1,-0.5,0.5])
    df_scaled = df_scaled.multiply(weights, axis=1)
    my_scores = df_scaled.mean(axis=1)
    scaler = MinMaxScaler()
    my_scores_scaled = scaler.fit_transform(my_scores.values.reshape(-1,1))*100
    return my_scores_scaled


def add_extrapolated_revenue(df):
    # now that info is finalized add the extrapolated revenue data
    svg_columns = ['monthly_revenue', 'nightly_revenue', 'monthly_occupancy']
    for col_name in svg_columns:
        # average of lists of history of svg
        avg, cv = get_history_extrapolations(df[col_name])
        df['avg_' + col_name] = avg
        # seasonality measure
        df['seasonality_' + col_name] = cv

    # the multiply by 0.3 comes from 30 days / 100 for occupancy to become a percent between 0-1
    df['expected_avg_monthly_revenue'] = (df['avg_nightly_revenue'].mul(
        df['avg_monthly_occupancy'])).apply(lambda x: x * .3)

    # divide revenue by rooms and guests to get average per room and per guest
    df['revenue_per_room'] = df['avg_monthly_revenue'].divide(df['rooms'])

    df['expected_revenue_per_room'] = df['expected_avg_monthly_revenue'].divide(
        df['rooms'])

    df['expected_revenue_per_guest'] = df[
        'expected_avg_monthly_revenue'].divide(df['guests'])

    df['listing_growth_percent_quarter'] = [
        float(i[0])
        for i in df.quarterly_growth.str.extract(r'((?:-\d)|(?:\d+))').values
    ]
    df['percent_full_time'] = [
        float(i)
        for i in df.pct_available_full_time.str.extract(r'(\d+)').values
    ]
    df['percent_entire_home_available'] = [
        float(i) for i in df.percent_entire_listing.str.extract(r'(\d+)').values
    ]
    df['rating_avg'] = [float(i) for i in df.rating_avg.values]

    df['num_zips_in_place'] = get_num_zip_codes(df)

    df['state'] = df.state_city_zip.str.extract('(.*?)/')

    df['city'] = add_city_column(df)

    # put in hud data where zri is missing
    df['Zri_hud'] = df[['Zri', 'adjusted_median_rent_hud'
                       ]].apply(lambda row: row['adjusted_median_rent_hud']
                                if np.isnan(row['Zri']) else row['Zri'],
                                axis=1)

    df['rent_to_rent_sqft'] = df.revenue_per_room.divide(
        df['ZriPerSqft_AllHomes2019-06'].apply(lambda x: x * 670))
    df['rent_to_rent_expected_sqft'] = df.expected_revenue_per_room.divide(
        df['ZriPerSqft_AllHomes2019-06'].apply(lambda x: x * 670))
    df['rent_to_rent'] = df.avg_monthly_revenue.divide(df['Zri_hud'])
    df['rent_to_rent_expected'] = df.expected_avg_monthly_revenue.divide(
        df['Zri_hud'])
    df['my_mean_rent_to_rent'] = df[[
        'rent_to_rent', 'rent_to_rent_expected', 'rent_to_rent_sqft',
        'rent_to_rent_expected_sqft'
    ]].mean(axis=1)

    # add distance to zip data
    df['distance_from_Eli'] = df[['longitude',
                                  'latitude']].apply(get_distance_from_nyc,
                                                     axis=1)
    
    df['annual_growth_rate'] = get_annual_change(df)
    
    df['score'] = get_scores(df) 


    return df


df = add_extrapolated_revenue(df)


/home/michael/anaconda3/envs/airbnb_analysis/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


### Filter for widgets and downloads

In [6]:
# filtered df for qgrid
filtered_df = df[[True if not isNaN(i) else False for i in df.avg_revenue_svg]]
filtered_df = filtered_df[[
    True if not isNaN(i) else False for i in filtered_df.legal_ranking
]]
filtered_df['score_f'] = get_scores(filtered_df)

# subset from df to use as qgrid frame
my_qgrid = filtered_df[[
    'state', 'county', 'city', 'zip', 'legal_ranking','current_active_listings', 'rooms',
    'expected_avg_monthly_revenue', 'seasonality_monthly_revenue',
    'my_mean_rent_to_rent', 'score_f', 'distance_from_Eli',
    'avg_monthly_occupancy', 'rating_avg', 'annual_growth_rate','pop','notes','longitude','latitude','finalists'
]]

# plt.rcParams['figure.figsize'] = [20, 15]
# my_plot=pd.plotting.scatter_matrix(my_qgrid[[
#         'current_active_listings', 'expected_avg_monthly_revenue',
#         'seasonality_monthly_revenue', 'my_mean_rent_to_rent',
#         'distance_from_Eli', 'avg_monthly_occupancy', 'rating_avg','annual_growth_rate','score_f'
#     ]])



### Write to file

In [7]:

my_qgrid.to_excel('../process/data/df.xlsx', index=False)
my_qgrid.to_pickle('../process/data/df.pkl')

In [194]:
#pd.set_option('display.max_rows', 10)
#my_filtered_grid